In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from matplotlib.font_manager import FontProperties
from abc import ABCMeta,abstractmethod
import pandas as pd
import numpy as np
from random import gauss
from math import exp, sqrt
import math
import datetime
import matplotlib.pyplot as plt
from WindPy import w
import Ipynb_importer
from Option import OrdinaryOption
w.start()

importing Jupyter notebook from Option.ipynb
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
class ValueOption(object):
    __metaclass__ = ABCMeta #指定这是一个抽象类
    @abstractmethod  #抽象方法
    def getValue(self):
        pass

In [5]:
class LongstaffPricingModel(ValueOption): 
    
    def __init__(self, option, discountFactor, stockPrices):
        self.option = option
        self.discountFactor = discountFactor
        self.stockPrices = stockPrices
        self.totalTradingDays = len(stockPrices[0])

    def getValue(self):
#         DISCOUNT_FACTOR = 0.94176
        self.strikingPrice = self.option.strikingPrice
        self.optionType = self.option.optionType
#         stockPrices = [[1, 1.09, 1.08, 1.34], 
#                       [1, 1.16, 1.26, 1.54], 
#                       [1, 1.22, 1.07, 1.03], 
#                       [1, 0.93, 0.97, 0.92], 
#                       [1, 1.11, 1.56, 1.52],
#                       [1, 0.76, 0.77, 0.9], 
#                       [1, 0.92, 0.84, 1.01],
#                       [1, 0.88, 1.22, 1.34]]
#         totalTradingDays = len(stockPrices[0])

        #一个二维数组，代表每条path每个时间点上，每一个点的行权收益
        optionPayOffData = self.getOptionPayOffData(self.stockPrices, self.option)

        #创建一个二维数组，用来存储每条path每个时间点上是否行权的信息，1为行权，0为不行权
        exersizeTimeTable = [[0 for i in range(self.totalTradingDays)] for i in range(len(self.stockPrices))]

        #创建一个二维数组，用来存储每条path每个时间点上的real cash flow
        cashFlowTable = [[0 for i in range(self.totalTradingDays)] for i in range(len(self.stockPrices))]
        ##########################################################################################################
        print("log1")
        #更新行权时间表中最后一列
        dateIndex = self.totalTradingDays - 1
        for pathIndex in range(0, len(optionPayOffData)):
            exerciseDayPayoff = optionPayOffData[pathIndex][dateIndex]
            if(exerciseDayPayoff > 0):
                exersizeTimeTable[pathIndex][dateIndex] = 1
        #更新CF table最后一列
        self.updateCashFlowTable(exersizeTimeTable, cashFlowTable, optionPayOffData)
        print("log2")
        #循环
        for dateIndex in range(self.totalTradingDays - 2, 0, -1):
            #check倒数第二天中，in the money的情况
            regression_X = []
            regression_Y = []
            regression_pathIndex = []
        #     dateIndex = totalTradingDays - 2       
            for pathIndex in range(0, len(optionPayOffData)):
                payoff = optionPayOffData[pathIndex][dateIndex]
                if(payoff > 0):
                    regression_pathIndex.append(pathIndex)
                    regression_X.append(self.stockPrices[pathIndex][dateIndex])
                    if(dateIndex == self.totalTradingDays - 2):
                        regression_Y.append((optionPayOffData[pathIndex][dateIndex + 1]) * self.discountFactor)
                    else:
                        regression_Y.append((cashFlowTable[pathIndex][dateIndex + 1]) * self.discountFactor)

            #regression：T3和T2
            regressionResult = self.leastSquaresRegression(regression_X, regression_Y, 2)        
            print(regressionResult)
            #根据regression结果，做出比较，若payOffAtCurrent>payOffContinuation则current行权，并更新exerciseTimeTable该点为1，该点后面所有1改为0，因为期权整个期间只能行权一次
            #并更新cash flow table
            for i in range(0, len(regression_Y)):
                pathIndex = regression_pathIndex[i]#在大表中的path index
                payOffContinuation = regression_X[i] * regression_X[i] * regressionResult[0] + regression_X[i] * regressionResult[1] + regressionResult[2]
                payOffAtCurrent = optionPayOffData[pathIndex][dateIndex]
                if(payOffAtCurrent > payOffContinuation):
                    exersizeTimeTable[pathIndex][dateIndex] = 1
                    self.updateFollowingsToZero(exersizeTimeTable, pathIndex, dateIndex)
                    self.updateCashFlowTable(exersizeTimeTable, cashFlowTable, optionPayOffData)
        # print(pd.DataFrame(exersizeTimeTable))   
        print(pd.DataFrame(cashFlowTable)) 


        #计算价格
        totalPrice = 0
        for rowIndex in range(0, len(exersizeTimeTable)):
            currentRow = exersizeTimeTable[rowIndex]
            for i in range(1, len(currentRow)):
                if(currentRow[i] == 1):
                    totalPrice = totalPrice + self.getDiscountedNumber(self.discountFactor, cashFlowTable[rowIndex][i], i)    
                    break
        optionPrice = totalPrice / len(self.stockPrices)
        return optionPrice
    
    def getOptionPayOffData(self, stockPrices, option):
        """
        获取期权所有蒙特卡洛模拟价格路径中，每个点的行权收益。
        返回：二维数组
        """
        #新建一个二维数组，与stockPrice size一样，用来存储期权收益数据
        optionPayoffData = [[0 for i in range(len(stockPrices[0]))] for i in range(len(stockPrices))]

        #遍历optionPayoffData所有数字更新每个数字为在该点行权的期权收益。
        for pathIndex in range(0, len(stockPrices)):
            for dateIndex in range(0, len(stockPrices[0])):
                optionPayoffData[pathIndex][dateIndex] = self.option.option_payoff(stockPrices[pathIndex][dateIndex])
        return optionPayoffData
    
    def leastSquaresRegression(self, x, y, highestPower):
        """
        对x，y最最小二乘
        输入：x，y 均为list
        输出：list, 为回归的结果的所有系数，顺序从高次到低次排列。
        """
        factors = np.polyfit(x, y, highestPower)
        return factors
    
    def updateCashFlowTable(self, whenToExerciseTable, cashFlowTable, payOffTable):
        for pathIndex in range(0, len(cashFlowTable)):
            for dateIndex in range(0, len(cashFlowTable[0])):
                if(whenToExerciseTable[pathIndex][dateIndex] == 1):
                    cashFlowTable[pathIndex][dateIndex] = payOffTable[pathIndex][dateIndex]
                else:
                    cashFlowTable[pathIndex][dateIndex] = 0
                    
    def updateFollowingsToZero(self, whenToExerciseTable, currentRowIndex, currentColumnIndex):
        currentRow = whenToExerciseTable[currentRowIndex]
        for i in range(currentColumnIndex + 1, len(currentRow)):
            if(currentRow[i] == 1):
                currentRow[i] = 0
                
    def getDiscountedNumber(self, discountFactor, originalNumber, dateIndex):
        discountResult = originalNumber
        for i in range(0, dateIndex):
            discountResult = discountResult * discountFactor
        return discountResult    

In [19]:
# """test"""
# option = OrdinaryOption("PUT", 1.1)
# stockPrices = [[1, 1.09, 1.08, 1.34], 
#               [1, 1.16, 1.26, 1.54], 
#               [1, 1.22, 1.07, 1.03], 
#               [1, 0.93, 0.97, 0.92], 
#               [1, 1.11, 1.56, 1.52],
#               [1, 0.76, 0.77, 0.9], 
#               [1, 0.92, 0.84, 1.01],
#               [1, 0.88, 1.22, 1.34]]
# DISCOUNT_FACTOR = 0.94176
# model = LongstaffPricingModel(option, DISCOUNT_FACTOR, stockPrices)
# model.getValue()

In [8]:
# def getDatesList(beginningDate, endingDate):
#     """
#     获取日期序列，return a list of dates

#     参数：
#         起始日期，终止日期
        
#     返回值：
#         这段日期的交易日，格式为list
#     """
#     #从wind获取datetime列表
#     w.isconnected()
    
#     #日期list
#     datesListInDatetime = w.tdays(beginningDate, endingDate, "").Data[0]
#     return datesListInDatetime

In [9]:
# def getStockData(startDate, endDate, stockCode):
#     """
#     获取某个指数某一段时间的收盘价格，返回值为WindData

#     参数：startDate：起始日期，字符串格式
#            endDate：终止日期，字符串格式
#          benchmark：需要查询的指数或者股票代码，字符串格式

#     返回值：WindData，e.g.
#           .ErrorCode=0
#           .Codes=[000905.SH]
#           .Fields=[CLOSE]
#           .Times=[20201201,20201202,20201203,20201204,20201207,20201208,20201209,20201210,20201211]
#           .Data=[[6426.6155,6434.9791,6440.8808,6464.7345,6424.6823,6423.6513,6307.2207,6320.0956,6209.427]]
#     """
#     w.isconnected()
#     closingData = w.wsd(stockCode, "close", startDate, endDate, "")
#     return closingData

In [5]:
# def option_payoff(flag, S_T, strkingPrice):
#     """期权收益"""
#     if flag.lower() == 'call':
#         return max(S_T - strkingPrice, 0.0)
#     else:
#         return max(strkingPrice - S_T, 0.0)

In [8]:
# regressor = LinearRegression()
# model = regressor.fit(X_train, y_train)

# x = np.linspace(0, 25, 25)
# #生成每个baix对应的y
# y = (model.coef_ * x + model.intercept_)[0]

# #画直线
# plt.plot(x, y, c='orange')
# plt.scatter(X_train, y_train,s = 40)

# # plt1.scatter(xx, yy,s = 40)
# # xx = np.linspace(0, 26, 5)
# # plt1.plot(xx, yy, label="linear equation")

In [9]:
# #多项式回归（本例中为二次回归）
# #首先生成多项式特征
# quadratic_featurizer = PolynomialFeatures(degree = 2)
# X_train_quadratic = quadratic_featurizer.fit_transform(X_train)
  
# regressor_quadratic = LinearRegression()
# regressor_quadratic.fit(X_train_quadratic, y_train)


# #给出一些点，并画出线性回归的曲线
# xx = np.linspace(0, 26, 100)
# #标准化
# xx_quadratic = quadratic_featurizer.transform(xx.reshape(xx.shape[0], 1))
# plt.plot(x, y, c='orange')
# plt.scatter(X_train, y_train,s = 40)
# plt.plot(xx, regressor_quadratic.predict(xx_quadratic), c = 'green')

In [10]:
# (xx.reshape(xx.shape[0], 1))
# xx.shape

# x = np.linspace(0, 25, 25)



##Longstaff文献中做regression的算法

In [23]:

# x = np.array([1.08, 1.07, 0.97, 0.77, 0.84])
# y = np.array([0, 0.06592320, 0.16951680, 0.18835200, 0.08475840])

# x1 = [1.08, 1.07, 0.97, 0.77, 0.84]
# y1 = [0, 0.06592320, 0.16951680, 0.18835200, 0.08475840]

# z1 = np.polyfit(x1, y1, 2)#3为多项式最高次幂，结果为多项式的各个系数
# #最高次幂3，得到4个系数,从高次到低次排列
# #最高次幂取几要视情况而定
# p1 = np.poly1d(z1)#将系数代入方程，得到函式p1
# print(z1)#多项式系数
# # print(p1)#多项式方程